<a href="https://colab.research.google.com/github/Salmaabidah/CrawlingTweeter/blob/main/Preprocessing_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Modul

In [ ]:
import warnings; warnings.simplefilter('ignore')
import nltk

try:
    import google.colab
    IN_COLAB = True
    !wget https://raw.githubusercontent.com/taudata-indonesia/eLearning/master/taudataDDGsna.py
    !mkdir data
    !wget -P data/ https://raw.githubusercontent.com/taudata-indonesia/eLearning/master/data/contoh.pdf
    !wget -P data/ https://raw.githubusercontent.com/taudata-indonesia/eLearning/master/data/slang.txt
    !wget -P data/ https://raw.githubusercontent.com/taudata-indonesia/eLearning/master/data/stopwords_id.txt
    !wget -P data/ https://raw.githubusercontent.com/taudata-indonesia/eLearning/master/data/stopwords_en.txt
    !wget -P data/ https://raw.githubusercontent.com/taudata-indonesia/eLearning/master/data/kata_dasar.txt
    !wget -P data/ https://raw.githubusercontent.com/taudata-indonesia/eLearning/master/data/wn-ind-def.tab
    !wget -P data/ https://raw.githubusercontent.com/taudata-indonesia/eLearning/master/data/wn-msa-all.tab
    !wget -P data/ https://raw.githubusercontent.com/taudata-indonesia/eLearning/master/data/ind_SA.csv
    !wget -P data/ https://raw.githubusercontent.com/taudata-indonesia/eLearning/master/data/all_indo_man_tag_corpus_model.crf.tagger
    !wget https://raw.githubusercontent.com/taudata-indonesia/eLearning/master/taudataNlpTm.py
    !wget -P data/ https://raw.githubusercontent.com/taudata-indonesia/eLearning/master/data/data_sna.csv
    !wget https://raw.githubusercontent.com/taudata-indonesia/eLearning/master/network_unp2021.gexf
    !wget https://raw.githubusercontent.com/taudata-indonesia/eLearning/master/taudata_sna.py
    !wget https://raw.githubusercontent.com/taudata-indonesia/eLearning/master/taudataEDA02CTA.py

    !pip install pyLDAvis
    !pip install spacy unidecode textblob sastrawi pyLDAvis
    !pip install --upgrade python-crfsuite gensim
    !pip install sklearn-pycrfsuite
    !pip install --upgrade spacy python-crfsuite unidecode textblob sastrawi sklearn-pycrfsuite
    !pip install --upgrade unidecode twython tweepy beautifulsoup4 tika
    !pip install unidecode twython tweepy beautifulsoup4 json_lines lxml pyLDAvis
    !pip install spacy python-crfsuite unidecode textblob sastrawi sklearn-pycrfsuite
    !python -m spacy download en
    !python -m spacy download xx
    !python -m spacy download en_core_web_sm
    !python -m spacy download xx_ent_wiki_sm
    !python -m spacy download xx_sent_ud_sm
    nltk.download('popular')

except:
    IN_COLAB = False
    print("Running the code locally, please make sure all the python module versions agree with colab environment and all data/assets downloaded")

In [ ]:
import nltk, warnings; warnings.simplefilter('ignore')
import pandas as pd, taudataNlpTm as tau
import taudataDDGsna as tau
import tweepy, json, urllib.request, requests
from urllib.request import Request, urlopen
from twython import TwythonStreamer
from bs4 import BeautifulSoup as bs
from tqdm import tqdm
from nltk.corpus import stopwords
"Done"

'Done'

##Crawling


In [ ]:
# Contoh API Keys (Sesuaikan dengan API keys masing-masing)
Ck = '' # consumer_key
Cs = '' # consumer_secret
At = '' # access_token
As = '' # access_secret


'Done'

'Done'

In [ ]:
import tweepy

auth = tweepy.auth.OAuthHandler(Ck, Cs)
auth.set_access_token(At, As)
api = tweepy.API(auth, wait_on_rate_limit=True, timeout=180, retry_count=5, retry_delay=3)
usr_ = api.verify_credentials()
print('Welcome "{}" you are now connected to twitter server'.format(usr_.name))

Welcome "Salma Abidah Faizah" you are now connected to twitter server


In [ ]:
qry = '("KAMPUS MERDEKA" OR "MBKM")'
N = 5000 # jumlah N tweet "terkini" yang ingin diambil
T = []
for post in tqdm(tweepy.Cursor(api.search_tweets, q=qry, lang='id', tweet_mode='extended').items(N)):
    T.append(post)


In [ ]:
T[0]._json

In [ ]:
tweet = [t._json for t in T]

## Menyimpan Crawling

In [ ]:
import json
def saveTweets(tweets, file='Tweets.json'): #in Json Format
    with open(file, 'w') as f:
        for t in tweets:
            try:
                f.write(json.dumps(t)+'\n')
            except:
                pass

In [ ]:
# Menyimpan hasil crawling twitter
fileName = 'data/Data.json'
saveTweets(tweet, file=fileName)
print('Saved to '+fileName)

# Load Kembali

In [ ]:
!wget -P data/ https://raw.githubusercontent.com/Salmaabidah/CrawlingTweeter/main/Data.json
fileName = 'data/Data.json'

--2023-07-28 02:24:28--  https://raw.githubusercontent.com/Salmaabidah/CrawlingTweeter/main/Data.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4692416 (4.5M) [text/plain]
Saving to: ‘data/Data.json’

Data.json           100%[===================>]   4.47M  --.-KB/s    in 0.1s    

2023-07-28 02:24:28 (46.5 MB/s) - ‘data/Data.json’ saved [4692416/4692416]



In [ ]:
def loadTweets(file='Tweets.json'):
    f=open(file,encoding='utf-8', errors ='ignore', mode='r')
    T=f.readlines();f.close()
    for i,t in enumerate(T):
        T[i] = json.loads(t.strip())
    return T

In [ ]:
Data_Tweet = loadTweets(file=fileName)

# Preprocessing

In [ ]:
df = pd.DataFrame((Data_Tweet))
df[['created_at', 'full_text', 'favorite_count', 'retweet_count']]

,created_at,full_text,favorite_count,retweet_count
0,Thu Jun 08 12:07:02 +0000 2023,cv nya phuwin\n- aktif bekerja di dunia hibura...,0,0
1,Thu Jun 08 12:06:06 +0000 2023,Kalamangga siluman laba laba yang hidup sejak ...,0,0
2,Thu Jun 08 11:56:52 +0000 2023,DI JADWAL UAS TERTULIS KALAU HARI RABU JAM 7.3...,0,0
3,Thu Jun 08 11:50:09 +0000 2023,SPILL FAKTA-FAKTA MBKM PENELITIAN/RISET DONG✨ ...,0,0
4,Thu Jun 08 11:48:34 +0000 2023,Pengen ikut mbkm atau pertukaran pelajar,0,0
...,...,...,...,...
1150,Tue May 30 11:19:08 +0000 2023,"gapapaaa deh pusing-pusing semester ini, asal ...",0,0
1151,Tue May 30 11:13:30 +0000 2023,"@mochiorenji Astungkaraaaa paling serius, tpi ...",0,0
1152,Tue May 30 11:06:24 +0000 2023,"@um_fess Dim nadim, ngurusi gojek gak bener ae...",10,0
1153,Tue May 30 10:50:03 +0000 2023,Bismillah semoga lolos magang kampus merdeka h...,0,0


In [ ]:
# ambil data text nya
df1 = df['full_text']
df1 = pd.DataFrame(df1)
df1.columns=['text']
df1

,text
0,cv nya phuwin\n- aktif bekerja di dunia hibura...
1,Kalamangga siluman laba laba yang hidup sejak ...
2,DI JADWAL UAS TERTULIS KALAU HARI RABU JAM 7.3...
3,SPILL FAKTA-FAKTA MBKM PENELITIAN/RISET DONG✨ ...
4,Pengen ikut mbkm atau pertukaran pelajar
...,...
1150,"gapapaaa deh pusing-pusing semester ini, asal ..."
1151,"@mochiorenji Astungkaraaaa paling serius, tpi ..."
1152,"@um_fess Dim nadim, ngurusi gojek gak bener ae..."
1153,Bismillah semoga lolos magang kampus merdeka h...


In [ ]:
# huruf kecil semua
df1['text'] = df1['text'].str.lower()

In [ ]:
# hapus retweets
import re

df2 = pd.DataFrame(df1)

def remove_retweets_mentions(tweet):
    # Menghapus retweet (format @username)
    tweet = re.sub(r'@[A-Za-z0-9_]+', '', tweet)

    # Menghapus mentions (format RT @username)
    tweet = re.sub(r'RT @[A-Za-z0-9_]+', '', tweet)

    return tweet.strip()

# Menggunakan metode apply untuk menerapkan fungsi remove_retweets_mentions pada kolom "text"
df2['notweet_text'] = df2['text'].apply(remove_retweets_mentions)

print(df2)


                                                   text  \
0     cv nya phuwin\n- aktif bekerja di dunia hibura...   
1     kalamangga siluman laba laba yang hidup sejak ...   
2     di jadwal uas tertulis kalau hari rabu jam 7.3...   
3     spill fakta-fakta mbkm penelitian/riset dong✨ ...   
4              pengen ikut mbkm atau pertukaran pelajar   
...                                                 ...   
1150  gapapaaa deh pusing-pusing semester ini, asal ...   
1151  @mochiorenji astungkaraaaa paling serius, tpi ...   
1152  @um_fess dim nadim, ngurusi gojek gak bener ae...   
1153  bismillah semoga lolos magang kampus merdeka h...   
1154  butuh joki cv ats buat kampus merdeka yang bis...   

                                           notweet_text  
0     cv nya phuwin\n- aktif bekerja di dunia hibura...  
1     kalamangga siluman laba laba yang hidup sejak ...  
2     di jadwal uas tertulis kalau hari rabu jam 7.3...  
3     spill fakta-fakta mbkm penelitian/riset dong✨ ...  
4

In [ ]:
# stopword & lemma

from tqdm import tqdm
stopId, lemmaId = tau.LoadStopWords(lang='id')
slangFixId = tau.loadCorpus(file = 'data/slang.txt', sep=':')

cleaned_data = []
for tweet in tqdm(df2['notweet_text']):
    cleaned_data.append(tau.cleanText(tweet, lemma=lemmaId, lan='id', stops = stopId, fix=slangFixId))
"done"


100%|██████████| 1155/1155 [05:35<00:00,  3.45it/s]


'done'

In [ ]:
print(Data_Tweet[0], end='\n\n') # sebelum clean
print(cleaned_data[0], end='\n\n') # setelah clean

{'created_at': 'Thu Jun 08 12:07:02 +0000 2023', 'id': 1666778863622553600, 'id_str': '1666778863622553600', 'full_text': 'cv nya phuwin\n- aktif bekerja di dunia hiburan sejak 2014\n- menjadi artis dan berperan di serial gmmtv sejak 2018\n- menyelesaikan magang kampus merdeka di agensi saya sendiri😭💔', 'truncated': False, 'display_text_range': [0, 176], 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': []}, 'metadata': {'iso_language_code': 'in', 'result_type': 'recent'}, 'source': '<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 1139759658103496704, 'id_str': '1139759658103496704', 'name': 'mas jamet | esd', 'screen_name': 'ssingssayaflu', 'location': '', 'description': 'shipper fangirl multiverse of 101% thai only ,akun rill min', 'url': None, 'enti

In [ ]:
df3 = pd.DataFrame((cleaned_data))
df3.columns=['full_text']
df3.head()

,full_text
0,cv nya phuwin aktif dunia hibur artis peran se...
1,kalamangga siluman laba laba hidup raja kuta k...
2,jadwal uas tulis rabu jam ganti bikin bingung ...
3,spill fakta mbkm teliti riset
4,mbkm tukar ajar


In [ ]:
# Menghapus duplikat
df3 = df3.drop_duplicates('full_text')

In [ ]:
#df3.to_csv('data/datalab.csv',index=False)

**Setelah didownload dilakukan tranlate menggunakan google translate dan deepl. Namun ternyata masih ada data yang belum bersih, sehingga perlu dilakukkan pembersihan secara lebih teliti lagi**

In [ ]:
df3 = df3.drop([1])
df3.head()

,full_text
0,cv nya phuwin aktif dunia hibur artis peran se...
2,jadwal uas tulis rabu jam ganti bikin bingung ...
3,spill fakta mbkm teliti riset
4,mbkm tukar ajar
5,jhaa urus mandiri kocak bolak urus mbkm nim bisa


In [ ]:
# Kamus untuk kata pengganti
replacement_dict = {
    "kocak": "lucu",
    "jhaa": "tertawa",
    "nder": "sender",
    "kaprodi": "ketua program studi",
    "ku": "saya",
    "rt":"retweet",
    "moga": "semoga",
    "deg deg an": "khawatir",
    "sempro": "seminar proposal",
    "metopen": "metode penelitian",
    "tempatkuu": "tempat saya",
    "smt": "semester",
    "pt": "perusahaan",
    "jt": "juta",
    "nyampe": "sampai",
    "nangiss": "nangis",
    "do": "di",
    "kakk": "kak",
    "fafifu": "banyak omong",
    "gasih": "engga",
    "gadibolehin": "tidak diperbolehkan",
    "gaboleh": "tidak diperbolehkan",
    "ae": "aja",
    "bngt": "banget",
    "gmna": "gimana",
    "kipk": "kartu indonesia pintar",
    "kamipasti": "kami pasti",
    "ikur": "ikut",
    "msib": "masih",
    "ben gendeng": "biar gila",
    "mutualan": "saling follow",
    "wa": "whatsapp",
    "dm": "direct message",
    "mbmm": "mbkm",
    "gaiiss": "guys",
    "infosumbar": "informasi sumatra barat",
    "didik": "pendidikan",
    "dongg" : "hanya",
    "dpn": "depan",
    "kmren": "kemarin"
}

# Fungsi untuk mengganti kata dalam teks berdasarkan kamus
def replace_words_perword(text, words_to_replace):
    words = text.split()
    for i, word in enumerate(words):
        if word.lower() in words_to_replace:
            words[i] = words_to_replace[word.lower()]
    return ' '.join(words)

# Menggunakan metode apply untuk menerapkan fungsi replace_words_perword pada kolom "full_text"
df3['processed_text'] = df3['full_text'].apply(replace_words_perword, args=(replacement_dict,))
df3.head()

,full_text,processed_text
0,cv nya phuwin aktif dunia hibur artis peran se...,cv nya phuwin aktif dunia hibur artis peran se...
2,jadwal uas tulis rabu jam ganti bikin bingung ...,jadwal uas tulis rabu jam ganti bikin bingung ...
3,spill fakta mbkm teliti riset,spill fakta mbkm teliti riset
4,mbkm tukar ajar,mbkm tukar ajar
5,jhaa urus mandiri kocak bolak urus mbkm nim bisa,tertawa urus mandiri lucu bolak urus mbkm nim ...


In [ ]:
# Kata-kata yang ingin dihapus
words_to_remove = ["gai", "ii", "dips", "ug", "ah", "zonabu", "kah", "ami", "wkwkwk", "bal", "um", "wkskxkskdj", "bi", "an", "ba", "wowkwowkkw",
                   "woi", "xixi", "wrwwrwrwr", "huhu", "?", "nic", " hng", "awokwokwok", "bro", "anj", "jirr", "kkkk", "mmm", "ckckck", "ti", "gp",
                   "uwiw", "zonauang"]

# Fungsi untuk menghapus kata-kata tertentu dalam teks
def remove_full_words(text, words_to_remove):
    for word in words_to_remove:
        pattern = r'\b{}\b'.format(word)
        text = text.replace(pattern, "")
    return text.strip()

# Menggunakan metode apply untuk menerapkan fungsi remove_words pada kolom "full_text"
df3['word_remove'] = df3['processed_text'].apply(remove_full_words, args=(words_to_remove,))
df3


,full_text,processed_text,word_remove
0,cv nya phuwin aktif dunia hibur artis peran se...,cv nya phuwin aktif dunia hibur artis peran se...,cv nya phuwin aktif dunia hibur artis peran se...
2,jadwal uas tulis rabu jam ganti bikin bingung ...,jadwal uas tulis rabu jam ganti bikin bingung ...,jadwal uas tulis rabu jam ganti bikin bingung ...
3,spill fakta mbkm teliti riset,spill fakta mbkm teliti riset,spill fakta mbkm teliti riset
4,mbkm tukar ajar,mbkm tukar ajar,mbkm tukar ajar
5,jhaa urus mandiri kocak bolak urus mbkm nim bisa,tertawa urus mandiri lucu bolak urus mbkm nim ...,tertawa urus mandiri lucu bolak urus mbkm nim ...
...,...,...,...
1150,gapapaa deh pusing semester semester terima ma...,gapapaa deh pusing semester semester terima ma...,gapapaa deh pusing semester semester terima ma...
1151,astungkaraa serius harap saing bngt pliss acco...,astungkaraa serius harap saing banget pliss ac...,astungkaraa serius harap saing banget pliss ac...
1152,dim nadim ngurusi gojek ae dadi tri an kampus ...,dim nadim ngurusi gojek aja dadi tri an kampus...,dim nadim ngurusi gojek aja dadi tri an kampus...
1153,bismillah moga lolos magang kampus merdeka,bismillah semoga lolos magang kampus merdeka,bismillah semoga lolos magang kampus merdeka


In [ ]:
# ambil data text nya
df4 = df3['word_remove']
df4

0       cv nya phuwin aktif dunia hibur artis peran se...
2       jadwal uas tulis rabu jam ganti bikin bingung ...
3                           spill fakta mbkm teliti riset
4                                         mbkm tukar ajar
5       tertawa urus mandiri lucu bolak urus mbkm nim ...
                              ...                        
1150    gapapaa deh pusing semester semester terima ma...
1151    astungkaraa serius harap saing banget pliss ac...
1152    dim nadim ngurusi gojek aja dadi tri an kampus...
1153         bismillah semoga lolos magang kampus merdeka
1154    butuh joki cv kampus merdeka beres besok via d...
Name: word_remove, Length: 929, dtype: object

In [ ]:
df4.to_csv('data/translate_data.csv',index=False)